# TPE INF4248 TAL/NLP
## GROUPE 4
### SUJET: STEMMATISATION

## 1.   PorterStemmer

### Description:
C'est l'une des méthodes de radicalisation les plus populaires proposées en 1980. Elle est basée sur l'idée que les suffixes de la langue anglaise sont constitués d'une combinaison de suffixes plus petits et plus simples.
Ce stemmer est connu pour sa rapidité et sa simplicité. Les principales applications de Porter Stemmer incluent l'exploration de données et la recherche d'informations. Cependant, ses applications ne sont limitées qu'aux mots anglais. De plus, le groupe de radicaux est mappé sur le même radical et le radical de sortie n'est pas nécessairement un mot significatif. Les algorithmes sont de nature assez longue et sont connus pour être les plus anciens stemmer.

Exemple : EED -> EE signifie « si le mot a au moins une voyelle et une consonne plus la terminaison EED, changez la terminaison en EE » car « agreed » devient « agree ».

### Avantages :
Il produit le meilleur résultat par rapport aux autres stemmers et il a moins de taux d'erreur.

### Inconvenients :
- Les variantes morphologiques produites ne sont pas toujours de vrais mots.

- PorterStemmer est l'un des meilleurs algorithmes de Stemmatisation.

In [ ]:
from nltk.stem import *
from nltk.tokenize import word_tokenize

Création d'une instance de PorterStemmer

In [ ]:
ps = PorterStemmer()

Initialisation de nos données pour tester le Stemmer

In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np

drive.mount('/content/drive')
url = '/content/drive/My Drive/INF4248/Dataset/stemming/EmotionHappy.csv'

df_happy = pd.read_csv(url)

# Suppression des doublons
df_happy = df_happy.drop_duplicates(subset=['content', 'sentiment'])
df_happy = df_happy.reset_index(drop=True)

print(df_happy.head())
print('\n')
print(df_happy.info())


Mounted at /content/drive
                                             content sentiment
0  Wants to know how the hell I can remember word...     happy
1  Love is a long sweet dream & marriage is an al...     happy
2  The world could be amazing when you are slight...     happy
3  My secret talent is getting tired without doin...     happy
4  Khatarnaak Whatsapp Status Ever… Can\’t talk, ...     happy


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 704 entries, 0 to 703
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   content    704 non-null    object
 1   sentiment  704 non-null    object
dtypes: object(2)
memory usage: 11.1+ KB
None


Pour nettoyer les données et créer des nuages ​​de mots, nous nous sommes inspiré du notebook suivant : https://www.kaggle.com/moezabid/disaster-tweets-nlp

In [ ]:
import re
import string
from textblob import TextBlob
from tqdm.notebook import tqdm

In [ ]:
contractions = {
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are",
"thx"   : "thanks"
}


def clean(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    text = text.lower()
    text = re.sub('\[.*?\]','',text)
    text = re.sub('https?://\S+|www\.\S+','',text)
    text = re.sub('<,*?>+"','',text)
    text = re.sub('[%s]' % re.escape(string.punctuation),'',text)
    text = re.sub('\n','',text)
    text = re.sub('\w*\d\w*','',text)
    text = re.sub("xa0'", '', text)
    text = re.sub(u"\U00002019", "'", text) # IMPORTANT: le caractère apostrophe n'était pas le caractère habituel...
    words = text.split()
    for i in range(len(words)):
        if words[i].lower() in contractions.keys():
            words[i] = contractions[words[i].lower()]
    text = " ".join(words)
    text = TextBlob(text).correct()
    return text

df_happy['content'] = df_happy['content'].apply(lambda x: clean(x))

# Suppression des lignes de données vides
df_happy['content'].replace('', np.nan, inplace=True)
df_happy = df_happy.dropna(subset = ['content'])
df_happy = df_happy.reset_index(drop=True)


Suppresssion des stopwords

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    text = text.split()
    words = [w for w in text if w not in stopwords.words('english')]
    return " ".join(words)

df_happy['content_no_sw'] = df_happy['content'].apply(lambda x : remove_stopwords(x))

Recupération du texte pour l'execution des Stemmer

In [ ]:
sentences = df_happy['content'].values.copy()
words_tk = []
for sentence in sentences:
        text = str(sentence)
        words_tk.append(word_tokenize(text))

words_for_stem = words_tk[0]
print(words_for_stem)

['wants', 'to', 'know', 'how', 'the', 'hell', 'i', 'can', 'remember', 'words', 'to', 'songs', 'from', 'years', 'ago', 'but', 'can', 'not', 'remember', 'what', 'i', 'went', 'into', 'the', 'next', 'room', 'for']


Execution de PorterStemmer sur nos données

In [ ]:
for word in words_for_stem:
    print(word, " : ", ps.stem(word))

wants  :  want
to  :  to
know  :  know
how  :  how
the  :  the
hell  :  hell
i  :  i
can  :  can
remember  :  rememb
words  :  word
to  :  to
songs  :  song
from  :  from
years  :  year
ago  :  ago
but  :  but
can  :  can
not  :  not
remember  :  rememb
what  :  what
i  :  i
went  :  went
into  :  into
the  :  the
next  :  next
room  :  room
for  :  for


#### Execution de PorterStemmer sur des mots où nous obtenons du sur-racinement (over-stemming) qui est une erreur de radicalisation, faisant référence à la situation où un stemmer produit une forme racine qui n'est pas un mot valide ou qui n'est pas la forme racine correcte d'un mot.

In [ ]:
words = ["language", "history", "historical", "article", "programmers", "poodle", "eaten", "mouse", "remember", "update", "easily", "leaves", "university", "fairly", "sportingly"]

for word in words:
  print(word, ":" , ps.stem(word))

language : languag
history : histori
historical : histor
article : articl
programmers : programm
poodle : poodl
eaten : eaten
mouse : mous
remember : rememb
update : updat
easily : easili
leaves : leav
university : univers
fairly : fairli
sportingly : sportingli


## 2. SnowBallStemmer

### Description:
Il s'agit d'un algorithme de stemming également connu sous le nom d'algorithme de stemming Porter2 car il s'agit d'une meilleure version du PorterStemmer puisque certains problèmes ont été résolus dans ce stemmer.

Example: sitting -> sitt -> sit

### Avantages :
Prise en charge de plusieurs langues
Elle est très rapide et peut gérer le retrait des lettres doubles dans des mots comme "getting" étant transformé en "get" et gère également les pluriels irréguliers comme 'teeth' et 'tooth' etc.

### Inconvenients :
Elle prend du temps et de nombreux suffixes ne sont pas disponibles dans le tableau des terminaisons. C'est très peu fiable.

In [ ]:
# Cet algorithme prend en charge plusieurs langues
print(" ".join(SnowballStemmer.languages))

arabic danish dutch english finnish french german hungarian italian norwegian porter portuguese romanian russian spanish swedish


Différence entre Porter Stemmer et Snowball Stemmer :

- Snowball Stemmer est plus agressif que Porter Stemmer.
- Prend en charge plusieurs langues.
- Il n'y a qu'une petite différence dans le fonctionnement de ces deux.
Des mots comme "fairly" et "sportingly" ont été réduits en « fair » et « sport » dans snowball Stemmer, mais lorsque vous utilisez Porter Stemmer ils sont réduits en « fairli » et « sportingli ».
La différence entre les deux algorithmes peut être clairement vue dans la façon dont le mot "sportingly" est contenu par les deux. Il est clair que Snowball Stemmer le ramène à une racine plus précise.

Execution de LancasterStemmer sur notre liste de mots contenu dans words

In [ ]:
snow = SnowballStemmer(language='english')
for word in words:
    print(word, ":", snow.stem(word))

language : languag
history : histori
historical : histor
article : articl
programmers : programm
poodle : poodl
eaten : eaten
mouse : mous
remember : rememb
update : updat
easily : easili
leaves : leav
university : univers
fairly : fair
sportingly : sport


## 3. Lancaster Stemmer

### Description:
C'est un algorithme itératif. Un tableau contenant environ cent vingt règles indexées par la dernière lettre d'un suffixe. Chaque règle spécifie soit la suppression soit le remplacement d'un suffixe. C'est simple et chaque itération prend soin des deux (suppression et remplacement) selon la règle appliquée.

### Avantages :
Simple et chaque itération prise en charge à la fois la suppression et le remplacement selon la règle appliquée.

### Inconvenients :
C'est un algorithme très lourd et le sur-racinement peut se produire.


In [ ]:
lancas = LancasterStemmer()

Execution de LancasterStemmer sur notre liste de mots contenu dans words

In [ ]:
for word in words:
    print(word, ":", lancas.stem(word))

language : langu
history : hist
historical : hist
article : artic
programmers : program
poodle : poodl
eaten : eat
mouse : mous
remember : rememb
update : upd
easily : easy
leaves : leav
university : univers
fairly : fair
sportingly : sport


**Remarque:** Lancaster contrairement au deux autres (Porter et Snowball) à bien radicalisé "programmers" en "program", "easily" en "easy" mais à pour la majorité du reste Lancaster a aussi mal radicalisé.

## 4. Regexp Stemmer

### Description:

Regex stemmer identifie les affixes morphologiques à l'aide d'expressions régulières. Les sous-chaînes correspondant aux expressions régulières seront ignorées.

On lui donne en parametre une ou plusieurs suffixe(s) qu'il supprimera dans à la fin de chaque mots qu'on lui passera et la taille minimal (min) qu'un mot doit avoir pour être radicalisé.

### Avantages:
On définit nous même les terminaisons ainsi que la taille des mots à radicaliser.

### Inconvenients:
Ne peut vraiment être appliqué à un très grand volume de nom varié;
Les mots obtenus ne seront forcément pas valide.


In [ ]:
regexp = RegexpStemmer('en$|s$|e$|able$|es$', min=4)

Execution de Regexp Stemmer sur notre liste de mots contenu dans words

In [ ]:
for word in words:
  print(word, ":", regexp.stem(word))

language : languag
history : history
historical : historical
article : articl
programmers : programmer
poodle : poodl
eaten : eat
mouse : mous
remember : remember
update : updat
easily : easily
leaves : leav
university : university
fairly : fairly
sportingly : sportingly


# Références:
 * https://medium.com/@ajay_khanna/different-techniques-of-stemming-a99d2fe8c08c [consulté le 26/03/2023 à 15h03]
 * https://www.ijrte.org/wp-content/uploads/papers/v8i4/C6200098319.pdf
[consulté le 26/03/2023 à 18h16]
 * https://www.analyticsvidhya.com/blog/2021/11/an-introduction-to-stemming-in-natural-language-processing/ [consulté le 28/03/2023 à 04h02]
 * https://www.geeksforgeeks.org/introduction-to-stemming/ [consulté le 30/03/2023 à 05h05]
 * https://www.kaggle.com/moezabid/disaster-tweets-nlp [consulté le 30/03/2023 à 06h09]

 Dataset:
 - (Emotion Happy) https://www.kaggle.com/code/ronikdedhia/emotion-whatsapp-chat/input/?select=Emotion%28happy%29.csv [téléchargé le 30/03/2023 04h25]
 - Crée à partir de données provenant de certains des sites présents en référence.
